In [1]:
import os 
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
     

In [13]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir =r"C:\food\pythonProject\image data\rice_leaf_diseases\train"
test_dir=r"C:\food\pythonProject\image data\rice_leaf_diseases\val"

In [17]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*")) 
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")


4 Classes
1192 Train images


In [18]:

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [19]:
input_shape=(256,256,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(256,256),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(256,256),batch_size=32)
     

Found 1192 images belonging to 4 classes.


Found 280 images belonging to 4 classes.


In [20]:
validation_generator = train_datagen.flow_from_directory(
                       test_dir, # same directory as training data
                       target_size=(256,256),
                       batch_size=32)

Found 280 images belonging to 4 classes.


In [21]:
learning_rate = 5e-5
train_batch_size = 256
eval_batch_size = 256
num_epochs = 30
num_classes = 4  # Replace with the actual number of classes in your dataset

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers up to a specific layer
freeze_until_layer = 'block4_conv3'
for layer in pretrained_model.layers:
    if layer.name == freeze_until_layer:
        break
    layer.trainable = False

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model2 = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    return lr * (1 - epoch / num_epochs)

lr_callback = LearningRateScheduler(lr_scheduler)

# Train the model
history = model2.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_callback]
)


Epoch 1/30
4/4 [==============================] - 55s 14s/step - loss: 0.8320 - accuracy: 0.6953 - val_loss: 1.6821 - val_accuracy: 0.7188 - lr: 5.0000e-05
Epoch 2/30
4/4 [==============================] - 55s 14s/step - loss: 0.3698 - accuracy: 0.9141 - val_loss: 2.1504 - val_accuracy: 0.5000 - lr: 4.8333e-05
Epoch 3/30
4/4 [==============================] - 39s 10s/step - loss: 0.3587 - accuracy: 0.8750 - val_loss: 1.0257 - val_accuracy: 0.6562 - lr: 4.5111e-05
Epoch 4/30
4/4 [==============================] - 39s 10s/step - loss: 0.2601 - accuracy: 0.9453 - val_loss: 1.6827 - val_accuracy: 0.4375 - lr: 4.0600e-05
Epoch 5/30
4/4 [==============================] - 39s 10s/step - loss: 0.1588 - accuracy: 0.9219 - val_loss: 1.8523 - val_accuracy: 0.5938 - lr: 3.5187e-05
Epoch 6/30
4/4 [==============================] - 39s 10s/step - loss: 0.1550 - accuracy: 0.9219 - val_loss: 1.2864 - val_accuracy: 0.6875 - lr: 2.9322e-05
Epoch 7/30
4/4 [==============================] - 40s 11s/step -

In [22]:
test_loss, test_accuracy = model2.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

9/9 [==============================] - 56s 6s/step - loss: 2.2289 - accuracy: 0.7071
Test Loss: 2.2289230823516846
Test Accuracy: 0.7071428298950195


In [24]:
model2.save(r"C:\food\pythonProject\rice.h5")